In [1]:
import pickle

import gensim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import twint
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


In [4]:
import os

dir = './emotions_prediction'
os.chdir("SocialMediaAnalysis")
print(os.getcwd())

with open("{}/data/dataset.raw.pickle".format(dir), "rb") as dataset_file:
    dataset = pickle.load(dataset_file, encoding='latin1')


/Users/r.makowiecki/PycharmProjects/SocialMediaAnalysis


In [5]:
labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']

# print dataset type and sample
print(type(dataset))
print(dataset['info'][:10])
print(dataset['texts'][:10])

<class 'dict'>
[{'label': array([1., 0., 0., 0., 0., 0., 0.])}, {'label': array([0., 1., 0., 0., 0., 0., 0.])}, {'label': array([0., 0., 1., 0., 0., 0., 0.])}, {'label': array([0., 0., 0., 1., 0., 0., 0.])}, {'label': array([0., 0., 0., 0., 1., 0., 0.])}, {'label': array([0., 0., 0., 0., 0., 1., 0.])}, {'label': array([0., 0., 0., 0., 0., 0., 1.])}, {'label': array([1., 0., 0., 0., 0., 0., 0.])}, {'label': array([0., 1., 0., 0., 0., 0., 0.])}, {'label': array([0., 0., 1., 0., 0., 0., 0.])}]
['During the period of falling in love, each time that we met and especially when we had not met for a long time.', 'When I was involved in a traffic accident.', 'When I was driving home after  several days of hard work, there was a motorist ahead of me who was driving at 50 km/hour and refused, despite his low speeed to let me overtake.', 'When I lost the person who meant the most to me.', "The time I knocked a deer down - the sight of the animal's injuries and helplessness.  The realization that t

In [6]:
X = dataset['texts']
Y = [item['label'] for item in dataset['info']]
    
#print labels sample
print(Y[:10])

[array([1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0.]), array([0., 0., 0., 0., 0., 0., 1.]), array([1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0.])]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [13]:
def get_english_stopwords():
    file_path = './emotions_prediction/data/nltk_english_stopwords'
    with open(file_path, 'r') as stop_words_file:
        return stop_words_file.read().splitlines()


In [14]:
stop_words = get_english_stopwords()
print(stop_words)

def vectorize_sentence(model_w2v, sentence, stop_words):
    words = gensim.utils.simple_preprocess(sentence)

    sentence_vector = []
    for word in words:
        if word not in stop_words and word in model_w2v.vocab:
            sentence_vector.append(model_w2v[word])
    sentence_vector = np.stack(sentence_vector, axis=0)

    return np.mean(sentence_vector, axis=0)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no